# Winton Stock Market Analysis

In [8]:
import numpy as np
import tensorflow as tf
import pandas as pd
import keras as K

from keras.layers import LSTM

## Preprocessing

In [71]:
train_raw = pd.read_csv('/dataset/winton-stock-market-challenge/train.csv')
test_raw = pd.read_csv('/dataset/winton-stock-market-challenge/test.csv')

In [77]:
train = train_raw.fillna(0)
train = train.drop(['Id'], axis=1)
train.describe()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Ret_175,Ret_176,Ret_177,Ret_178,Ret_179,Ret_180,Ret_PlusOne,Ret_PlusTwo,Weight_Intraday,Weight_Daily
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,...,40000.000000,4.000000e+04,4.000000e+04,4.000000e+04,40000.000000,40000.000000,40000.000000,40000.000000,4.000000e+04,4.000000e+04
mean,0.600200,-0.090678,0.541124,0.327286,5.482775,0.410145,49244.971525,0.194649,10.179650,2.435100,...,0.000049,-3.580980e-06,-9.675766e-06,-1.268935e-05,-0.000017,-0.000057,-0.000210,0.000012,1.504600e+06,1.880750e+06
std,1.761757,1.086873,0.893417,0.735459,2.942324,1.464542,28242.409717,0.139294,3.583524,2.451191,...,0.001115,1.166019e-03,1.248956e-03,1.217967e-03,0.001208,0.001270,0.025039,0.024160,2.059143e+05,2.573928e+05
min,0.000000,-3.440521,-4.643526,-5.440596,1.000000,-0.936644,338.000000,0.000000,0.000000,0.000000,...,-0.018048,-1.786752e-02,-3.526027e-02,-4.864099e-02,-0.037215,-0.058046,-0.627690,-0.450779,1.000035e+06,1.250044e+06
25%,0.000000,-0.791707,-0.092559,0.000000,2.000000,-0.250258,26143.000000,0.016600,8.000000,0.000000,...,-0.000316,-3.943210e-04,-4.091851e-04,-3.649676e-04,-0.000361,-0.000413,-0.010521,-0.010055,1.357070e+06,1.696337e+06
50%,0.000000,0.000000,0.398252,0.159130,6.000000,0.014157,48457.000000,0.213800,10.000000,1.000000,...,0.000004,-4.039616e-07,4.016833e-07,6.407432e-07,0.000001,-0.000002,-0.000258,-0.000258,1.477713e+06,1.847142e+06
75%,0.000000,0.091542,1.037897,0.804371,8.000000,0.523780,72387.000000,0.331800,12.000000,5.000000,...,0.000407,3.820685e-04,4.172179e-04,3.750965e-04,0.000378,0.000364,0.010005,0.009772,1.624389e+06,2.030486e+06
max,10.000000,4.175150,4.530405,2.953163,10.000000,12.609885,99861.000000,0.365000,36.000000,6.000000,...,0.020781,3.061460e-02,4.406460e-02,2.922928e-02,0.048221,0.026112,0.795602,0.303038,2.761229e+06,3.451536e+06


## Model